In [1]:
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout,AveragePooling2D
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import *
from tensorflow import keras
from tensorflow.keras import layers

def download_model():
    model = Sequential()

    conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
    model.add(conv_base)
    model.add(BatchNormalization())
    model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='softmax'))

    train_layers = [layer for layer in conv_base.layers[::-1][:5]]

    for layer in conv_base.layers:
        if layer in train_layers:
            layer.trainable = True
    model.save("model/model.h5")

In [2]:
def load_model():
    download_model()
    model=tf.keras.models.load_model("model/model.h5", compile=False)
    model.compile(optimizer =tf.keras.optimizers.Adam(learning_rate=0.00001,decay=0.0001),metrics=["accuracy"],
                  loss= tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1))
    model.load_weights("C:/Users/shahe/Desktop/MED-SMART STARTUP PLAN/models/mammography.h5")
    return model

In [3]:
model = load_model()

In [4]:
import numpy as np 
from numpy import asarray
import cv2

class_name=['Benign with Density=1','Malignant with Density=1',
            'Benign with Density=2','Malignant with Density=2',
            'Benign with Density=3','Malignant with Density=3',
            'Benign with Density=4','Malignant with Density=4']
  

def preprocess(image):
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    im = cv2.filter2D(image, -1, kernel)
    return im

def predict_img(img):
    img=preprocess(img)
    img=img/255.0
    im=img.reshape(-1,224,224,3)
    pred=model.predict(im)[0]
    return {class_name[i]:float(pred[i]) for i in range(8)}

img = "C:/Users/shahe/Desktop/mammography/test/Image_25.jpg"
from PIL import Image
image = Image.open(img)
data = asarray(image)

predict_img(data)

{'Benign with Density=1': 0.015827601775527,
 'Malignant with Density=1': 0.04457152262330055,
 'Benign with Density=2': 0.013294555246829987,
 'Malignant with Density=2': 0.819430410861969,
 'Benign with Density=3': 0.022421279922127724,
 'Malignant with Density=3': 0.014057686552405357,
 'Benign with Density=4': 0.0494217686355114,
 'Malignant with Density=4': 0.0209752656519413}